# Tool Usage Example

This notebook demonstrates how to create and use **Tools** with an agent in MiniAutoGen. We will show how an agent can autonomously decide when to use a tool to answer a question.

The workflow will be as follows:
1. Define a custom tool (`CalculatorTool`).
2. Create an agent and associate the tool with it.
3. Build a pipeline that includes the tool selection and execution components.
4. Start a conversation where the agent uses the tool to solve a problem.

## 1. Installation and Setup

In [ ]:
# If you are running this locally, install the dependencies
!pip install miniautogen openai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load your environment variables (especially OPENAI_API_KEY)
load_dotenv()

# Define your OpenAI key here if you are not using a .env file
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = "sk-your-key-here"

## 2. Defining a Custom Tool

Let's create a simple tool that works like a calculator. It inherits from the `Tool` base class and implements the `execute` method.

In [ ]:
from miniautogen.tools import Tool

class CalculatorTool(Tool):
    def __init__(self):
        super().__init__(
            name="calculator",
            description="A simple calculator that can perform addition, subtraction, multiplication, and division. Takes two numbers (a, b) and an operator."
        )

    def execute(self, a: float, b: float, operator: str) -> str:
        try:
            a = float(a)
            b = float(b)
            if operator == '+':
                result = a + b
            elif operator == '-':
                result = a - b
            elif operator == '*':
                result = a * b
            elif operator == '/':
                if b == 0:
                    return "Error: Division by zero."
                result = a / b
            else:
                return f"Error: Unknown operator '{operator}'."
            return str(result)
        except ValueError:
            return "Error: Invalid number input."
        except Exception as e:
            return f"An unexpected error occurred: {e}"

## 3. Setting Up the Chat Environment and Pipeline

Now, let's set up all the necessary components for the conversation.

In [ ]:
from miniautogen.llms.llm_client import LiteLLMClient
from miniautogen.agent.agent import Agent
from miniautogen.chat.chat import Chat
from miniautogen.chat.chatadmin import ChatAdmin
from miniautogen.pipeline.pipeline import Pipeline
from miniautogen.pipeline.components import (
    UserResponseComponent,
    AgentReplyComponent,
    TerminateChatComponent,
    Jinja2SingleTemplateComponent,
    LLMResponseComponent,
    NextAgentSelectorComponent,
    ToolSelectionComponent,  # New component
    ToolExecutionComponent   # New component
)

# LLM Client
litellm_client = LiteLLMClient(default_model='gpt-4-turbo-preview')

# Tool
calculator = CalculatorTool()

# Agents
user_agent = Agent("user", "User", "A human user.", pipeline=Pipeline([UserResponseComponent()]))
math_agent = Agent(
    "math_expert", 
    "MathExpert", 
    "An AI expert that can solve math problems, using a calculator if necessary.",
    tools=[calculator] # Associating the tool with the agent
)

# Chat
chat = Chat()
chat.add_agent(user_agent)
chat.add_agent(math_agent)

### Creating the Template and Pipeline for the Math Agent

The Jinja2 template is the most crucial part here. It needs to be able to handle the tool's output and include it in the final prompt for the LLM. This gives the LLM the context of the tool's result so it can formulate the final answer.

In [ ]:
tool_aware_template = """
[
  {"role": "system", "content": "{{ agent.role }}"},
  {% for message in messages %}
    {% if message.sender_id == agent.agent_id %}
      {"role": "assistant", "content": {{ message.message | tojson | safe }}}
    {% else %}
      {"role": "user", "content": {{ message.message | tojson | safe }}}
    {% endif %}
  {% endfor %}
  
  {# If a tool was called, add the result as context for the LLM #}
  {% if tool_output %}
    ,{"role": "assistant", "content": "(I have used the '{{ tool_call.name }}' tool and got the result: {{ tool_output }}. Now I will formulate the final answer.)"}
  {% endif %}
]
"""

# Template Component
jinja_component = Jinja2SingleTemplateComponent()
jinja_component.set_template_str(tool_aware_template)

# Math Agent Pipeline
math_agent_pipeline = Pipeline([
    ToolSelectionComponent(litellm_client, model_name='gpt-4-turbo-preview'),
    ToolExecutionComponent(),
    jinja_component,
    LLMResponseComponent(litellm_client)
])

math_agent.pipeline = math_agent_pipeline

## 4. Running the Conversation

Now let's set up the `ChatAdmin` and start the conversation. Ask a question that requires a calculation.

In [ ]:
# Administrator Pipeline
admin_pipeline = Pipeline([
    NextAgentSelectorComponent(), 
    AgentReplyComponent(), 
    TerminateChatComponent()
])

# ChatAdmin
chat_admin = ChatAdmin(
    "admin",
    "Admin",
    "The admin of the chat.",
    admin_pipeline,
    chat,
    "Solve the user's request and terminate.",
    max_rounds=4,
)

# Add an initial message to start
chat.add_message('user', "What is 125.5 multiplied by 4?")

# Run the chat
chat_admin.run()

### Observing the Output

When you run the cell above, you should see logs in the console. Notice the following:
1. `INFO:root:LLM decided to call tool: calculator` - The `ToolSelectionComponent` decided to use the calculator.
2. `INFO:root:Executing tool 'calculator' with arguments: {'a': 125.5, 'b': 4, 'operator': '*'}` - The `ToolExecutionComponent` runs the tool with the correct arguments.
3. The `MathExpert` then prints the final answer, which should be something like: `125.5 multiplied by 4 is equal to 502.0.`

This proves that the end-to-end workflow is functioning as expected.